In [1]:
import ccxt
import time
import pandas as pd
import json

In [2]:
with open('api.json') as f:
    apf = json.load(f)

In [3]:
bybit = ccxt.bybit({"apiKey":apf["api_key"], "secret":apf["secret_key"]})

In [9]:
#何秒ごとに価格を収集するか
interval = 5

#何BTC分取引するか
quantity = 0.001

In [10]:
#25個で計算した短期移動平均と75個で計算した長期移動平均のクロスを判定に使う
short_sma_duration = 25
long_sma_duration = 75

In [11]:
ticker_info = bybit.fetch_ticker(symbol = 'BTCUSDT')
ticker_info['last']

36011.0

In [12]:
dif = 250
dp=120
dl=120

In [17]:
ticker_info = bybit.fetch_ticker(symbol = 'BTCUSDT')
order = bybit.create_order('BTCUSDT','market','buy',quantity,0,{'take_profit': ticker_info['last']+dp,'stop_loss': ticker_info['last']-dl})

In [18]:
def price_data_collecting(samples=100):
    prices = []
    for _ in range(samples):
        ticker_info = bybit.fetch_ticker(symbol = 'BTCUSDT')
        last_price = ticker_info['last']
        prices.append(last_price)
        time.sleep(interval)
        return prices

In [ ]:
initial_data = price_data_collecting()
df = pd.DataFrame()
df["BTCUSDT"] = initial_data

while True:
    #最新価格を取得
    ticker_info = bybit.fetch_ticker(symbol = 'BTCUSDT')
    last_price = ticker_info['last']
    df=df.append({'BTCUSDT': last_price,}, ignore_index=True)

    #移動平均を計算する
    df["short_sma"]=df["BTCUSDT"].rolling(short_sma_duration).mean()
    df["long_sma"]=df["BTCUSDT"].rolling(long_sma_duration).mean()

    #ゴールデンクロス（短期移動平均が長期移動平均を下からつきぬける）
    if df["short_sma"].iloc[-1]>df["long_sma"].iloc[-1] and df["short_sma"].iloc[-2]<df["long_sma"].iloc[-2]:
        print("ロングポジションを取ります。")
        order = bybit.create_order('BTCUSDT','market','buy',quantity,0,{'take_profit': ticker_info['last']+dp,'stop_loss': ticker_info['last']-dl})

    #デッドクロス(ゴールデンクロスの逆)
    elif df["short_sma"].iloc[-1]<df["long_sma"].iloc[-1] and df["short_sma"].iloc[-2]>df["long_sma"].iloc[-2]:
        print("ショートポジションを取ります。")
        order = bybit.create_order('BTCUSDT','market','sell',quantity,0,{'take_profit': ticker_info['last']-dp,'stop_loss': ticker_info['last']+dl})

    #先頭行を削除してdfの長さを一定に保つ（長時間の運用時のメモリ対策）
    df=df.drop(df.index[0])

    time.sleep(interval)